# Проект спринта 7. «Секреты Темнолесья».

- Автор: Холкин Егор
- Дата: 7.03.2025

### Цели и задачи проекта

Цель проекта — познакомиться с данными, проверить их корректность и провести предобработку, получив необходимый срез данных.
Задача заключается в анализе и представлении данных о продажах игр, популярных платформах и предпочтениях игроков разных регионов. Статья должна привлечь внимание любителей старых игр и заинтересовать их проектом «Секреты Темнолесья».

### Описание данных

- Name — название игры.

- Platform — название платформы.

- Year of Release — год выпуска игры.

- Genre — жанр игры.

- NA sales — продажи в Северной Америке (в миллионах проданных копий).

- EU sales — продажи в Европе (в миллионах проданных копий).

- JP sales — продажи в Японии (в миллионах проданных копий).

- Other sales — продажи в других странах (в миллионах проданных копий).

- Critic Score — оценка критиков (от 0 до 100).

- User Score — оценка пользователей (от 0 до 10).

- Rating — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

### Содержимое проекта

В ходе проекта мы познакомились с данными, проверили их корректность и провели предобработку, получив необходимый срез данных. 
Конкретнее, были выполнены следующие задачи:

- [Данные обработаны на предмет устранения ошибок и пропусков](##проверка-ошибок-в-данных-и-их-предобработка)
  
- [Обнаружены и удалены дубликаты](###явные-и-неявные-дубликаты-в-данных)

- [Отобраны данные по времени выхода игры](##фильтрация-данных)

- [Категоризовать игры по оценкам пользователей и экспертов](#категоризация-данных)

- [Выделен топ-7 платформ по количеству игр, выпущенных за весь требуемый период](###выделим-топ-7-платформ-по-количеству-игр,-выпущенных-за-весь-актуальный-период.)

_(к сожалению, оглавление не работает у меня, хотя сделал вроде верно. Распространенная проблема среди моих однокурсников)_

---

## 1. Загрузка данных и знакомство с ними

Загружаем необходимые библиотеки и данные:

In [1]:
import pandas as pd

df = pd.read_csv('new_games.csv')


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


- Датасет представлен 11 столбцами и 16956 строками. Большая часть данных имеет тип Object, обоснованность чего мы проверим позднее. Имена столбцов написаны в классической стилистике, что является предпосылкой для перевода их в более удобный вид (например, snake case)
- Сразу обращаем внимание на наличие пропусков в данных, в столбце 8 - серьезные пропуски.

---

## Проверка ошибок в данных и их предобработка


### Названия, или метки, столбцов датафрейма

Приведем названия столбцов к Snake case:

In [3]:
df.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

In [4]:
df.columns = df.columns.str.replace(' ', '_').str.lower()
print(df.columns)


Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')


### Типы данных

Поработаем с типами данных. Для начала ознакомимся с уникальными значениями в основных столбцах:

In [5]:
# проверим уникальные значения в столбцах с пропусками

for col in ['year_of_release', 'eu_sales', 'jp_sales', 'user_score', 'critic_score', 'rating']:
    uniq = df[col].unique()
    print(f"Уникальные значения в столбце {col}: \n {uniq}")

Уникальные значения в столбце year_of_release: 
 [2006. 1985. 2008. 2009. 1996. 1989. 1984. 2005. 1999. 2007. 2010. 2013.
 2004. 1990. 1988. 2002. 2001. 2011. 1998. 2015. 2012. 2014. 1992. 1997.
 1993. 1994. 1982. 2016. 2003. 1986. 2000.   nan 1995. 1991. 1981. 1987.
 1980. 1983.]
Уникальные значения в столбце eu_sales: 
 ['28.96' '3.58' '12.76' '10.93' '8.89' '2.26' '9.14' '9.18' '6.94' '0.63'
 '10.95' '7.47' '6.18' '8.03' '4.89' '8.49' '9.09' '0.4' '3.75' '9.2'
 '4.46' '2.71' '3.44' '5.14' '5.49' '3.9' '5.35' '3.17' '5.09' '4.24'
 '5.04' '5.86' '3.68' '4.19' '5.73' '3.59' '4.51' '2.55' '4.02' '4.37'
 '6.31' '3.45' '2.81' '2.85' '3.49' '0.01' '3.35' '2.04' '3.07' '3.87'
 '3.0' '4.82' '3.64' '2.15' '3.69' '2.65' '2.56' '3.11' '3.14' '1.94'
 '1.95' '2.47' '2.28' '3.42' '3.63' '2.36' '1.71' '1.85' '2.79' '1.24'
 '6.12' '1.53' '3.47' '2.24' '5.01' '2.01' '1.72' '2.07' '6.42' '3.86'
 '0.45' '3.48' '1.89' '5.75' '2.17' '1.37' '2.35' '1.18' '2.11' '1.88'
 '2.83' '2.99' '2.89' '3.27' '2.22' '

Переходим к изменению типа данных у числовых столбцов на тип float64:
Обратим внимание на встречающиеся значения `unknown`. Вероятно, так обозначены пропуски в данных. Эти значения вызовут ошибку при применении функции `to_numeric`, поэтому используем параметр errors='coerce', который заменит `unknown` на NaN:



In [6]:
df['eu_sales'] = pd.to_numeric(df['eu_sales'], errors='coerce')
df['jp_sales'] = pd.to_numeric(df['jp_sales'], errors='coerce')
df['user_score'] = df['user_score'].replace('tbd', 'nan')  # заменим странное значение на nan
df['user_score'] = pd.to_numeric(df['user_score'], errors='coerce')
df['critic_score'] = pd.to_numeric(df['critic_score'], errors='coerce')

In [7]:
# проверим изменение типов данных:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16956 non-null  object 
 2   year_of_release  16681 non-null  float64
 3   genre            16954 non-null  object 
 4   na_sales         16956 non-null  float64
 5   eu_sales         16950 non-null  float64
 6   jp_sales         16952 non-null  float64
 7   other_sales      16956 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       7688 non-null   float64
 10  rating           10085 non-null  object 
dtypes: float64(7), object(4)
memory usage: 1.4+ MB


### Наличие пропусков в данных

- Посчитакм количество пропусков в каждом столбце в абсолютных и относительных значениях.


In [8]:
# в абсолютных:
print(df.isnull().sum())

name                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64


In [9]:
# в относительных:
print(df.isnull().mean())

name               0.000118
platform           0.000000
year_of_release    0.016218
genre              0.000118
na_sales           0.000000
eu_sales           0.000354
jp_sales           0.000236
other_sales        0.000000
critic_score       0.513918
user_score         0.546591
rating             0.405225
dtype: float64


Мы видим пропуски в столбцах critic_score (51%), user_score (55%) - скорее всего пользователи и критики не выставили свои оценки. Также видим пропуски в rating(40%) - они говорят о том, что есть незаполненные данные о возрастных категориях. 

Пропуски можно заполнить медианой или модой, удалить, или заменить индикаторами.



### Обработаем пропущенные значения

Удалим строки с пропусками в годе издания (таких всего 1,6%, так что можем ими пренебречь), жанре (<1%) и названии (<1%)

In [10]:
df = df.dropna(subset=['genre', 'name', 'year_of_release'])

Пропуски в critic_score и user_score мы заполним индикаторами (-1), так как процент пропусков слишком велик, чтобы заменять их средним значением. Позднее, при категоризации - выбросы от средних значений помешают нам.

In [11]:
# заполняем пропуски
df['critic_score'] = df['critic_score'].fillna(-1)
df['user_score'] = df['user_score'].fillna(-1)

- Пропуски в rating мы проигнорируем, так как значения данного столбца не понадобятся нам в дальнейшем анализе.
- Кстати, удалив пропуски в столбце даты релиза, можем произвести замену типа данных на `int64` для корректного представления.

In [12]:
df['year_of_release'] = df['year_of_release'].astype(int)

Пропуски в количествах продаж (eu и jp sales) заменим средним значением в зависимости от платформы и года выхода игры:

In [13]:
# подсчитаем средние значения кол-ва копий в разрезе платформы и года выхода
sales_avg = df.groupby(['platform', 'year_of_release'])[['eu_sales', 'jp_sales']].mean()
# напишем функцию, подставляющую средние значения вместо пропускоы
def average_copies(row, sales_avg):
    if pd.isna(row['eu_sales']):
        row['eu_sales'] = sales_avg.loc[(row['platform'], row['year_of_release']), 'eu_sales']
    if pd.isna(row['jp_sales']):
        row['jp_sales'] = sales_avg.loc[(row['platform'], row['year_of_release']), 'jp_sales']
    return row
# применим функцию к строкам
df = df.apply(average_copies, axis=1, sales_avg=sales_avg)

# проверим, заполнились ли пропуски корректно
eu_sales_missing = df['eu_sales'].isnull().sum()
jp_sales_missing = df['jp_sales'].isnull().sum()
print(f"Количество пропусков в 'eu_sales': {eu_sales_missing}")
print(f"Количество пропусков в 'jp_sales': {jp_sales_missing}")

Количество пропусков в 'eu_sales': 0
Количество пропусков в 'jp_sales': 0


### Явные и неявные дубликаты в данных

- Изучим уникальные значения в категориальных данных, например с названиями жанра игры, платформы, рейтинга и года выпуска. Проверим, встречаются ли среди данных неявные дубликаты, связанные с опечатками или разным способом написания.

In [14]:
# изучим уникальные значения в столбцах с категориальными данными

for col in ['year_of_release', 'platform', 'genre', 'name']:
    uniq = df[col].unique()
    print(f"Уникальные значения в столбце {col}: \n {uniq}")

Уникальные значения в столбце year_of_release: 
 [2006 1985 2008 2009 1996 1989 1984 2005 1999 2007 2010 2013 2004 1990
 1988 2002 2001 2011 1998 2015 2012 2014 1992 1997 1993 1994 1982 2016
 2003 1986 2000 1995 1991 1981 1987 1980 1983]
Уникальные значения в столбце platform: 
 ['Wii' 'NES' 'GB' 'DS' 'X360' 'PS3' 'PS2' 'SNES' 'GBA' 'PS4' '3DS' 'N64'
 'PS' 'XB' 'PC' '2600' 'PSP' 'XOne' 'WiiU' 'GC' 'GEN' 'DC' 'PSV' 'SAT'
 'SCD' 'WS' 'NG' 'TG16' '3DO' 'GG' 'PCFX']
Уникальные значения в столбце genre: 
 ['Sports' 'Platform' 'Racing' 'Role-Playing' 'Puzzle' 'Misc' 'Shooter'
 'Simulation' 'Action' 'Fighting' 'Adventure' 'Strategy' 'MISC'
 'ROLE-PLAYING' 'RACING' 'ACTION' 'SHOOTER' 'FIGHTING' 'SPORTS' 'PLATFORM'
 'ADVENTURE' 'SIMULATION' 'PUZZLE' 'STRATEGY']
Уникальные значения в столбце name: 
 ['Wii Sports' 'Super Mario Bros.' 'Mario Kart Wii' ...
 'Woody Woodpecker in Crazy Castle 5' 'LMA Manager 2007'
 'Haitaka no Psychedelica']


In [15]:
# приведем жанры и платформы к нижнему регистру

df['genre'] = df['genre'].str.lower()
df['platform'] = df['platform'].str.lower()

In [16]:
# изучим уникальные значения в столбцах с категориальными данными

for col in ['year_of_release', 'platform', 'genre', 'name']:
    uniq = df[col].unique()
    print(f"Уникальные значения в столбце {col}: \n {uniq}")

Уникальные значения в столбце year_of_release: 
 [2006 1985 2008 2009 1996 1989 1984 2005 1999 2007 2010 2013 2004 1990
 1988 2002 2001 2011 1998 2015 2012 2014 1992 1997 1993 1994 1982 2016
 2003 1986 2000 1995 1991 1981 1987 1980 1983]
Уникальные значения в столбце platform: 
 ['wii' 'nes' 'gb' 'ds' 'x360' 'ps3' 'ps2' 'snes' 'gba' 'ps4' '3ds' 'n64'
 'ps' 'xb' 'pc' '2600' 'psp' 'xone' 'wiiu' 'gc' 'gen' 'dc' 'psv' 'sat'
 'scd' 'ws' 'ng' 'tg16' '3do' 'gg' 'pcfx']
Уникальные значения в столбце genre: 
 ['sports' 'platform' 'racing' 'role-playing' 'puzzle' 'misc' 'shooter'
 'simulation' 'action' 'fighting' 'adventure' 'strategy']
Уникальные значения в столбце name: 
 ['Wii Sports' 'Super Mario Bros.' 'Mario Kart Wii' ...
 'Woody Woodpecker in Crazy Castle 5' 'LMA Manager 2007'
 'Haitaka no Psychedelica']


Неявных дубликатов в списке жанров, плафторм и года издания **не обнаружено**. Для удобства приведем названия игр также к нижнему регистру.

In [17]:
df['name'] = df['name'].str.lower()

**Займемся поисками явных дубликатов.** 
 Сначала по всему датафрейму:

In [18]:
duplicates = df[df.duplicated()]
display(duplicates)


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
268,batman: arkham asylum,ps3,2009,action,2.24,1.31,0.07,0.61,91.0,8.9,T
368,james bond 007: agent under fire,ps2,2001,shooter,1.90,1.13,0.10,0.41,72.0,7.9,T
717,god of war: ascension,ps3,2013,action,1.23,0.63,0.04,0.35,80.0,7.5,M
823,wipeout: the game,wii,2009,misc,1.94,0.00,0.00,0.12,-1.0,-1.0,NaN
848,rayman raving rabbids: tv party,wii,2008,misc,0.72,1.08,0.00,0.23,73.0,7.7,E10+
...,...,...,...,...,...,...,...,...,...,...,...
16671,fullmetal alchemist: prince of the dawn,wii,2009,adventure,0.00,0.00,0.01,0.00,-1.0,-1.0,NaN
16753,routes pe,ps2,2007,adventure,0.00,0.00,0.01,0.00,-1.0,-1.0,NaN
16799,transformers: prime,wii,2012,action,0.00,0.01,0.00,0.00,-1.0,-1.0,NaN
16912,metal gear solid v: the definitive experience,xone,2016,action,0.01,0.00,0.00,0.00,-1.0,-1.0,M


Обнаружилось 235 строк - явных дубликатов. Удалим их и сохраним результат в новый датафрейм:

In [19]:
df2 = df.drop_duplicates()

Подсчитаем процент удалённых в предыдущих шагах строк от изначального количества:

In [20]:
# увы, изначально при удалении строк перезаписывал df, поэтому теперь придется еще раз прочитать csv в новую переменную для расчета
df_start = pd.read_csv('new_games.csv')
print(round((1 - len(df2) / len(df_start))*100, 2), end=' % строк удалено')

3.02 % строк удалено

Таким образом, мы провели предобработку данных: 
- Привели названия и типы столбцов к корректным форматам.
- Обработали пропуски, выставив индикаторы, удалив, либо заменив средними значениями.

---

## Фильтрация данных

Для изучения истории продаж игр в начале XXI века, сделаем срез датафрейма с датой выхода игры с 2000 по 2013 год включительно. Сохраним новый срез данных в отдельном датафрейме - `df_actual`.

In [21]:
# фильтруем по дате релиза
df_actual = df2[(df2['year_of_release'] >= 2000) & (df2['year_of_release'] <= 2013)]

In [22]:
# проверим корректность среза
print(df_actual['year_of_release'].max())
print(df_actual['year_of_release'].min())

2013
2000


---

## Категоризация данных
    
### Проведем категоризацию данных:
- Разделим все игры:
-  по оценкам пользователей: высокая оценка (от 8 до 10), средняя оценка (от 3 до 8) и низкая оценка (от 0 до 3).
-  по оценкам критиков: высокая оценка (от 80 до 100), средняя оценка (от 30 до 80) и низкая оценка (от 0 до 30).

In [23]:
# сделаем копию df, чтоб питон не ругался
df_actual = df_actual.copy()

In [24]:
# категоризируем
df_actual.loc[:, 'Оценка пользователей'] = pd.cut(df_actual['user_score'], bins=[-2, 0, 3, 8, 10], right=False, labels=['нет оценки', 'низкая оценка','средняя оценка','высокая оценка'])
df_actual.loc[:, 'Оценка критиков'] = pd.cut(df_actual['critic_score'], bins=[-2, 0, 30, 80, 100], right=False, labels=['нет оценки', 'низкая оценка','средняя оценка','высокая оценка'])
df_actual.head()  
# предупреждение возникает из-за того, что мы дали label значению nan. Это может помешать дальнейшей работе с данными, поэтому так делать нельзя
# но в рамках данной задачи, думпю, можно закрыть глаза на это

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,Оценка пользователей,Оценка критиков
0,wii sports,wii,2006,sports,41.36,28.96,3.77,8.45,76.0,8.0,E,высокая оценка,средняя оценка
2,mario kart wii,wii,2008,racing,15.68,12.76,3.79,3.29,82.0,8.3,E,высокая оценка,высокая оценка
3,wii sports resort,wii,2009,sports,15.61,10.93,3.28,2.95,80.0,8.0,E,высокая оценка,высокая оценка
6,new super mario bros.,ds,2006,platform,11.28,9.14,6.50,2.88,89.0,8.5,E,высокая оценка,высокая оценка
7,wii play,wii,2006,misc,13.96,9.18,2.93,2.84,58.0,6.6,E,средняя оценка,средняя оценка


- После категоризации данных проверим результат: сгруппируем данные по выделенным категориям и посчитаем количество игр в каждой категории.

In [25]:
df_grouped = df_actual.groupby('Оценка пользователей', observed=False, dropna=False)['name'].count()
df_grouped


Оценка пользователей
нет оценки        6298
низкая оценка      116
средняя оценка    4081
высокая оценка    2286
Name: name, dtype: int64

In [26]:
df_grouped = df_actual.groupby('Оценка критиков', observed=False, dropna=False)['name'].count()
df_grouped

Оценка критиков
нет оценки        5612
низкая оценка       55
средняя оценка    5422
высокая оценка    1692
Name: name, dtype: int64

### Выделим топ-7 платформ по количеству игр, выпущенных за весь актуальный период.

In [27]:
#считаем кол-во игр
df_grouped_platforms = df_actual.groupby('platform')['name'].count()
#сортируем по убыванию и отбираем 7 значений
top_7_platforms = df_grouped_platforms.sort_values(ascending=False).head(7)
top_7_platforms

platform
ps2     2127
ds      2120
wii     1275
psp     1180
x360    1121
ps3     1087
gba      811
Name: name, dtype: int64

---

## Итоговый вывод


Итак, в ходе предобработки данных мы:
- Создали срез данных (игры, изданные с 2000 по 2013 год) и добавили классификацию по 
- Категоризовали игры по оценкам: низким, средним и высоким.
- Определили платформы, на которых было выпущено наибольшее количество игр в указанный период. Эти данные помогут в дальнейшем анализировать предпочтения игроков и влияние платформ на успех игр.

Полученные результаты могут быть использованы для более глубокого анализа рыночных трендов в игровой индустрии и для прогнозирования будущих успехов игр в зависимости от платформ и их оценки.

